# Import libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import zipfile
import re
import io

# Functions

In [2]:
def get_response(url):

    response = requests.get(url)
    response.raise_for_status()

    return response

In [3]:
def get_compiled_pattern(response, pattern):

    soup = BeautifulSoup(response.text, "html.parser")

    return re.compile(pattern), soup

In [4]:
def get_last_folder(url):

    response = get_response(url)

    pattern, soup = get_compiled_pattern(response, r"^\d{4}-\d{2}/$")

    folders = [a["href"].strip("/") for a in soup.find_all("a", href=pattern)]
    last_folder = sorted(folders)[-1]

    print("Última pasta encontrada:", last_folder)

    return last_folder

In [5]:
def get_source_data(
    endpoint,
    key,
    file_columns,
    file_dtypes
):

    last_updated_folder = get_last_folder(endpoint)

    response_folder = get_response(endpoint + last_updated_folder + "/")

    key_pattern, soup_folder = get_compiled_pattern(response_folder, r"^{key}\d+\.zip$".format(key=key))

    files = [a["href"] for a in soup_folder.find_all("a", href=key_pattern)]
    first_file = sorted(files, key=lambda x: int(re.search(r"(\d+)", x).group()))[0]
    print("Primeiro arquivo encontrado:", first_file)

    last_file = get_response(endpoint + last_updated_folder + "/" + first_file)

    z = zipfile.ZipFile(io.BytesIO(last_file.content))

    csv_name = z.namelist()[0]  # pega o primeiro arquivo dentro do zip
    print("Arquivo dentro do ZIP:", csv_name)

    df = pd.read_csv(
        z.open(csv_name),
        sep=";",
        encoding="latin1",
        usecols=list(range(len(file_columns))),
        names=file_columns,
        dtype=file_dtypes,
        header=None,
        low_memory=False
    )

    print("Total de linhas:", len(df))

    return df    

In [6]:
def check_for_duplicates(df):

    duplicates = df.duplicated(keep=False).sum()

    if duplicates > 0:
        print(f"Existem {duplicates} registros duplicados")
    else:
        print("Não existem CNPJs duplicados")

# Read Data

In [7]:
endpoint = "https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/"

## Empresas

In [8]:
def get_empresas(endpoint):

    company_columns = [
        "cnpj",
        "razao_social",
        "natureza_juridica",
        "qualificacao_responsavel",
        "capital_social",
        "cod_porte"
    ]

    company_dtypes = {
        "cnpj": "string",
        "razao_social": "string",
        "natureza_juridica": "Int64", 
        "qualificacao_responsavel": "Int64",
        "capital_social": "string",
        "cod_porte": "string"
    }

    df_empresas = (
        get_source_data(
            endpoint,
            "Empresas",
            company_columns,
            company_dtypes
        )
    )

    return df_empresas


df_empresas = get_empresas(endpoint)


Última pasta encontrada: 2025-08
Primeiro arquivo encontrado: Empresas0.zip
Arquivo dentro do ZIP: K3241.K03200Y0.D50809.EMPRECSV
Total de linhas: 23537416


## Sócios

In [9]:
def get_socios(endpoint):

    partners_columns = [
        "cnpj",
        "tipo_socio",
        "nome_socio",
        "documento_socio",
        "codigo_qualificacao_socio"
    ]

    partners_dtypes = {
        "cnpj": "string",
        "tipo_socio": "Int64",
        "nome_socio": "string", 
        "documento_socio": "string",
        "codigo_qualificacao_socio": "string"
    }

    df_socios = (
        get_source_data(
            endpoint,
            "Socios",
            partners_columns,
            partners_dtypes
        )
    )

    return df_socios


df_socios = get_socios(endpoint)

Última pasta encontrada: 2025-08
Primeiro arquivo encontrado: Socios0.zip
Arquivo dentro do ZIP: K3241.K03200Y0.D50809.SOCIOCSV
Total de linhas: 8024952


# Transform

## Empresas

In [10]:
# Drop Duplicates
df_empresas.drop_duplicates(inplace=True)

check_for_duplicates(df_empresas)

Não existem CNPJs duplicados


In [11]:
# Alterar campo capita_social para float
df_empresas["capital_social"] = (
    df_empresas["capital_social"]
    .str.replace(".", "", regex=False)   # remove pontos
    .str.replace(",", ".", regex=False)  # troca vírgula por ponto
    .astype(float)                       # converte para float
)

## Sócios

In [12]:
# Drop Duplicates
df_socios.drop_duplicates(inplace=True)

check_for_duplicates(df_socios)

Não existem CNPJs duplicados


In [13]:
# Check Null values for nome_socio
df_socios[df_socios["nome_socio"].isna()]

,cnpj,tipo_socio,nome_socio,documento_socio,codigo_qualificacao_socio
193222,93316370,2,<NA>,***000000**,16
199243,93536100,2,<NA>,***000000**,16
199519,00116136,2,<NA>,***000000**,16
200919,93594539,2,<NA>,***186130**,16
202878,00127405,2,<NA>,***155103**,16
...,...,...,...,...,...
6319273,51848851,2,<NA>,***000000**,16
6332915,03968617,2,<NA>,***000000**,16
6353954,78196516,2,<NA>,***000000**,16
6354104,13224258,2,<NA>,***000000**,16


In [ ]:
# Check Null values como dummies for documento_socio
df_socios[df_socios["documento_socio"] == "***000000**"]

,cnpj,tipo_socio,nome_socio,documento_socio,codigo_qualificacao_socio
193222,93316370,2,<NA>,***000000**,16
199243,93536100,2,<NA>,***000000**,16
199519,00116136,2,<NA>,***000000**,16
230432,94628328,2,CLAUDIOMIRO BICA PAIVA,***000000**,49
245391,95290342,2,<NA>,***000000**,16
...,...,...,...,...,...
6353954,78196516,2,<NA>,***000000**,16
6354104,13224258,2,<NA>,***000000**,16
6354140,92410018,2,<NA>,***000000**,16
6727365,15704499,2,CARLA ELIZA LIMA SOARES,***000000**,22


In [16]:
# Removendo socios sem identificaçao
df_socios = df_socios[
    ~(
        ((df_socios["nome_socio"].isna()) & (df_socios["documento_socio"].isna())) |
        ((df_socios["nome_socio"].isna()) & (df_socios["documento_socio"] == "***000000**"))
    )
].copy()

In [19]:
# Create foreigner flag
df_socios["flag_socio_estrangeiro"] = (
    df_socios["documento_socio"]
    .fillna("")  # substitui NA por string vazia
    .eq("***999999**")  # compara com o valor desejado
    .astype(int)  # converte True/False para 1/0
)

# Checando se funcionou
df_socios[(df_socios["flag_socio_estrangeiro"] == 1) & (df_socios["documento_socio"] != "***999999**")]

,cnpj,tipo_socio,nome_socio,documento_socio,codigo_qualificacao_socio,flag_socio_estrangeiro


In [20]:
# Agrupando por CNPJ qtd_socios e qtd_estrangeiros
df_socios_processed = df_socios.groupby("cnpj", dropna=False).agg(
    qtd_socios=("documento_socio", "count"),           # total de sócios
    qtd_estrangeiros=("flag_socio_estrangeiro", "sum") # total de estrangeiros
).reset_index()

# Relations

In [22]:
df_result = (
    ((df_empresas[["cnpj", "cod_porte"]]).drop_duplicates())
    .merge(
        df_socios_processed.drop_duplicates(),
        "inner",
        "cnpj"
    )
)

In [23]:
df_result["doc_alvo"] = df_result.apply(
    lambda row: True if row["cod_porte"] == "03" and row["qtd_socios"] > 1 else False,
    axis=1
)

In [24]:
# Criar flag booleana: True se qtd_estrangeiros > 0, False caso contrário
df_result["flag_socio_estrangeiro"] = df_result["qtd_estrangeiros"] > 0

In [25]:
df_result = df_result[["cnpj", "qtd_socios", "flag_socio_estrangeiro", "doc_alvo"]].drop_duplicates()

## DF final

In [26]:
df_result

,cnpj,qtd_socios,flag_socio_estrangeiro,doc_alvo
0,41273629,1,False,False
1,41273630,1,False,False
2,41273631,1,False,False
3,41273667,1,False,False
4,41273676,1,False,False
...,...,...,...,...
1579637,98756349,2,False,False
1579638,98759046,1,False,False
1579639,98818214,1,False,False
1579640,98819402,2,False,False


In [29]:
# Checando duplicados
duplicates = df_result["cnpj"].duplicated(keep=False).sum()

if duplicates > 0:
    print(f"Existem {duplicates} registros duplicados")
else:
    print("Não existem CNPJs duplicados")

Não existem CNPJs duplicados
